In [2]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import numpy as np
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier, plot_importance
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
import warnings
warnings.simplefilter('ignore')

<frozen importlib._bootstrap>:241: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject


- Submission_1 : Single XGB with random state 0 with SMOTE
- Submission_2 : Combination of 5 XGB and 5 Random Forest with Random state 0 with SMOTE
- Submission_3 : Combination of 5 Random Forest with Random state 0 with SMOTE
- Submission_4 : Submission 3 with Annual Income Dropped
- Submission_5 : Submission 2 with Annual Income Dropped
- Submission_6 : Submission 3 with Annual Income, Stay Duration of Current Residence and Tenure of Oldest Credit Line Dropped and random states 10 everywhere

In [3]:
train = pd.read_csv('TrainingData.csv')
test = pd.read_csv('testX.csv')

df = pd.concat([train, test], axis = 0)

df.shape

(130000, 53)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 130000 entries, 0 to 46999
Data columns (total 53 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   application_key  130000 non-null  int64  
 1   mvar1            130000 non-null  object 
 2   mvar2            120787 non-null  float64
 3   mvar3            129171 non-null  float64
 4   mvar4            129171 non-null  float64
 5   mvar5            129171 non-null  float64
 6   mvar6            130000 non-null  object 
 7   mvar7            130000 non-null  object 
 8   mvar8            130000 non-null  object 
 9   mvar9            130000 non-null  object 
 10  mvar10           130000 non-null  object 
 11  mvar11           130000 non-null  object 
 12  mvar12           130000 non-null  object 
 13  mvar13           130000 non-null  object 
 14  mvar14           130000 non-null  int64  
 15  mvar15           130000 non-null  object 
 16  mvar16           130000 non-null  objec

In [5]:
columns = ['ID',
           'Credit Worthiness Score',
           'Number and Riskiness of Credit Enquiry',
           'Default Severity (Any Loan)',
           'Default Severity (Auto Loan)',
           'Default Severity (Education Loan)',
           'Min Cred Available (Revolving Credit Cards)',
           'Max Cred Available (Active Credit Lines)',
           'Max Cred Available (Active Revolving Credit Cards)',
           'Total Available Credit (Cards with 1 missed payment)',
           'Total Credit Available',
           'Dues Collected Post Default',
           'Total Amount Due (Active Cards)',
           'Credit Amount Paid (Prev Year)',
           'Annual Income',
           'Property Value',
           'Full Credit Utilisation (Revolving Credit Cards)',
           'Full Credit Utilisation (Credit Cards)',
           'Full Credit Utilisation (Credit Lines)',
           '>75 percent Credit Utilisation (Credit Cards)',
           '>75 percent Credit Utilisation (Credit Lines)',
           'Average Utilisation (Revolving Credit Cards)',
           'Average Utilisation Last 2 Years (All Credit Lines)',
           'Average Utilisation Last Year (All Credit Cards)',
           'Average Utilisation Last 6 Months with 1 Missed Payment (Credit Cards)',
           'Average Tenure (Active Revolving Credit Cards)',
           'Tenure of Oldest Active Credit Card',
           'Tenure of Oldest Active Revolving Credit Card',
           'Days Since Last Missed Payment',
           'Tenure of Oldest Credit Line',
           'Max Tenure (Auto Loans)',
           'Max Tenure (Education Loans)',
           'Sum of Tenures (Active Credit Cards)',
           'Stay Duration of Current Residence',
           'Credit Lines with 1 Missed Payment (Last 6 Months)',
           'Revolving Credit Cards with 1 Missed Payment (Last 2 Years)',
           'Active Credit Lines',
           'Credit Cards with >2Y Tenure',
           'Credit Lines activated Prev 2 Years',
           'Credit Lines with Current Delinquency',
           'Utilization on Active Education Loans',
           'Utilization on Active Auto Loans',
           'Financial Stress Index',
           'Credit Lines with No Missed Payments (Prev 2 Years)',
           'Ratio(Max Amount Due : Sum of Amounts Due)',
           'Mortgage Loans (With 2 Missed Payements)',
           'Auto Loans (With 2 Missed Payments)',
           'Type of Product',
           'Int Value for an Application',
           'Bucketized Credit Worthiness Score',
           'Compound of mvar49 and mvar48',
           'mvar51',
           'default_ind']

In [6]:
df.columns = columns

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 130000 entries, 0 to 46999
Data columns (total 53 columns):
 #   Column                                                                  Non-Null Count   Dtype  
---  ------                                                                  --------------   -----  
 0   ID                                                                      130000 non-null  int64  
 1   Credit Worthiness Score                                                 130000 non-null  object 
 2   Number and Riskiness of Credit Enquiry                                  120787 non-null  float64
 3   Default Severity (Any Loan)                                             129171 non-null  float64
 4   Default Severity (Auto Loan)                                            129171 non-null  float64
 5   Default Severity (Education Loan)                                       129171 non-null  float64
 6   Min Cred Available (Revolving Credit Cards)                          

### Feature Understanding (Explanations and Stuff to Look into)

- Auto Loans: Loans where Vehicles are kept as collateral
- Revolving Credit: You have a fixed amount that you can borrow in some fixed time frame. After that the amount you can borrow replenishes to the full amount. Although you will have a maximum limit. For example, you can borrow 100 rupees a month max and the overall limit on that credit line is 500 rupees. So if your overall borrowed amount is 460 rupees and for that month you havenn't borrowed anything so far, you will be able to borrow only 40 rupees even though the monthly limit is 100
- Difference between Credit Worthiness Score and its bucketized version?
- Difference in statistics between train and test (Need for Adverserial Validation?)

### Feature Ideas and Experiments
- Binary Variable for checking if the person has moved into current residence after issuing any credit lines
- Credit Amount Paid / Number of Credit Lines with No Missed Payments
- Max Cred Available vs Default Severity
- Separate modeling for 0 Default Severity vs Non-Zero Default Severity
- KNN Missing Values Fill In
- Property Value vs Annual Income (Some Indicator of whether or not a person spends too much for their Income)
- 75% utilisation / Full Utilisation

In [8]:
df.head(10)

,ID,Credit Worthiness Score,Number and Riskiness of Credit Enquiry,Default Severity (Any Loan),Default Severity (Auto Loan),Default Severity (Education Loan),Min Cred Available (Revolving Credit Cards),Max Cred Available (Active Credit Lines),Max Cred Available (Active Revolving Credit Cards),Total Available Credit (Cards with 1 missed payment),Total Credit Available,Dues Collected Post Default,Total Amount Due (Active Cards),Credit Amount Paid (Prev Year),Annual Income,Property Value,Full Credit Utilisation (Revolving Credit Cards),Full Credit Utilisation (Credit Cards),Full Credit Utilisation (Credit Lines),>75 percent Credit Utilisation (Credit Cards),>75 percent Credit Utilisation (Credit Lines),Average Utilisation (Revolving Credit Cards),Average Utilisation Last 2 Years (All Credit Lines),Average Utilisation Last Year (All Credit Cards),Average Utilisation Last 6 Months with 1 Missed Payment (Credit Cards),Average Tenure (Active Revolving Credit Cards),Tenure of Oldest Active Credit Card,Tenure of Oldest Active Revolving Credit Card,Days Since Last Missed Payment,Tenure of Oldest Credit Line,Max Tenure (Auto Loans),Max Tenure (Education Loans),Sum of Tenures (Active Credit Cards),Stay Duration of Current Residence,Credit Lines with 1 Missed Payment (Last 6 Months),Revolving Credit Cards with 1 Missed Payment (Last 2 Years),Active Credit Lines,Credit Cards with >2Y Tenure,Credit Lines activated Prev 2 Years,Credit Lines with Current Delinquency,Utilization on Active Education Loans,Utilization on Active Auto Loans,Financial Stress Index,Credit Lines with No Missed Payments (Prev 2 Years),Ratio(Max Amount Due : Sum of Amounts Due),Mortgage Loans (With 2 Missed Payements),Auto Loans (With 2 Missed Payments),Type of Product,Int Value for an Application,Bucketized Credit Worthiness Score,Compound of mvar49 and mvar48,mvar51,default_ind
0,230032,1696,1.6541,0.000,0.0,0.0,0,6015,322,40369,18414,missing,6423,3067,123875,missing,1,1,1,3,3,94.78,8987.18,NaN,72.25,1462,4532,2890,61,4532,1095,3376,625,1.1667,1,0,8,10,4,1,73.78,82.547,0.08696,10,0.63899,na,0,C,10,770,4,3080,0.0
1,230033,1846,0.8095,0.000,0.0,0.0,102,7532,3171,18234,13664,missing,765,1931,42613,missing,0,0,0,0,0,74.25,953.06,953.06,4.80,1028,2099,2099,30386,2281,missing,2251,169,0.4167,0,0,8,0,2,0,99.129,missing,0,13,0.63836,na,na,L,732,437,5,2185,1.0
2,230034,1745,0.4001,0.000,0.0,0.0,missing,2536,missing,missing,2536,missing,missing,missing,76109,missing,na,na,na,0,0,NaN,NaN,NaN,NaN,missing,missing,missing,669,4623,3772,missing,missing,25.0833,0,na,1,na,1,0,missing,29.29,0,1,1.00000,na,0,C,89,795,4,3180,1.0
3,230035,1739,0.2193,0.000,0.0,0.0,1982,26440,4955,20316,37013,missing,0,0,84235,missing,0,0,0,0,0,NaN,0.00,NaN,0.00,1308,2525,791,91,5992,missing,3741,215,10.3333,0,0,3,3,2,0,96.272,missing,0.15385,3,0.53241,0,0,L,3,755,4,3020,0.0
4,230036,1787,0.0118,0.225,0.0,0.0,5451,5494,5494,7987,4696,missing,2257,27815,123875,524848,0,0,0,0,0,20.51,796.67,NaN,28.72,801,2281,2281,487,2707,missing,1947,158,0.9167,0,0,2,3,2,0,115.019,missing,0,1,0.92665,na,na,L,5,425,4,1700,0.0
5,230037,1579,NaN,3.502,0.0,0.0,missing,missing,missing,missing,0,199,missing,missing,77298,missing,na,na,na,0,0,NaN,NaN,NaN,NaN,missing,missing,missing,183,973,missing,missing,missing,1.3333,0,na,2,na,2,na,missing,missing,1.5,0,NaN,na,na,C,35,455,4,1820,1.0
6,230038,1818,0.4001,0.000,0.0,0.0,missing,1088,missing,1536,1498,missing,448,2331,123875,missing,na,0,0,0,0,NaN,NaN,NaN,29.16,1034,760,missing,30386,3437,missing,791,34,9.3333,0,na,2,0,0,0,88.171,missing,0,2,0.87224,na,0,C,2,392,5,1960,1.0
7,230039,na,NaN,NaN,NaN,NaN,missing,missing,missing,missing,missing,missing,missing,missing,198200,1263525,na,na,na,0,na,NaN,NaN,NaN,NaN,missing,missing,missing,missing,missing,missing,missing,missing,0.0833,na,na,na,na,na,na,missing,missing,missing,na,NaN,na,na,C,2,615,#VALUE!,#VALUE!,0.0
8,230040,1836,0.1358,0.000,0.0,0.0,347,38964,17828,70729,65843,missing,16298,18877,109010,missing,0,0,0,1,1,32.02,0.00,0.00,

In [9]:
df.replace('missing', np.nan, inplace = True)
df.replace('na', np.nan, inplace = True)
df.replace('#VALUE!', np.nan, inplace = True)

In [10]:
df.head(10)

,ID,Credit Worthiness Score,Number and Riskiness of Credit Enquiry,Default Severity (Any Loan),Default Severity (Auto Loan),Default Severity (Education Loan),Min Cred Available (Revolving Credit Cards),Max Cred Available (Active Credit Lines),Max Cred Available (Active Revolving Credit Cards),Total Available Credit (Cards with 1 missed payment),Total Credit Available,Dues Collected Post Default,Total Amount Due (Active Cards),Credit Amount Paid (Prev Year),Annual Income,Property Value,Full Credit Utilisation (Revolving Credit Cards),Full Credit Utilisation (Credit Cards),Full Credit Utilisation (Credit Lines),>75 percent Credit Utilisation (Credit Cards),>75 percent Credit Utilisation (Credit Lines),Average Utilisation (Revolving Credit Cards),Average Utilisation Last 2 Years (All Credit Lines),Average Utilisation Last Year (All Credit Cards),Average Utilisation Last 6 Months with 1 Missed Payment (Credit Cards),Average Tenure (Active Revolving Credit Cards),Tenure of Oldest Active Credit Card,Tenure of Oldest Active Revolving Credit Card,Days Since Last Missed Payment,Tenure of Oldest Credit Line,Max Tenure (Auto Loans),Max Tenure (Education Loans),Sum of Tenures (Active Credit Cards),Stay Duration of Current Residence,Credit Lines with 1 Missed Payment (Last 6 Months),Revolving Credit Cards with 1 Missed Payment (Last 2 Years),Active Credit Lines,Credit Cards with >2Y Tenure,Credit Lines activated Prev 2 Years,Credit Lines with Current Delinquency,Utilization on Active Education Loans,Utilization on Active Auto Loans,Financial Stress Index,Credit Lines with No Missed Payments (Prev 2 Years),Ratio(Max Amount Due : Sum of Amounts Due),Mortgage Loans (With 2 Missed Payements),Auto Loans (With 2 Missed Payments),Type of Product,Int Value for an Application,Bucketized Credit Worthiness Score,Compound of mvar49 and mvar48,mvar51,default_ind
0,230032,1696,1.6541,0.000,0.0,0.0,0,6015,322,40369,18414,NaN,6423,3067,123875,NaN,1,1,1,3,3,94.78,8987.18,NaN,72.25,1462,4532,2890,61,4532,1095,3376,625,1.1667,1,0,8,10,4,1,73.78,82.547,0.08696,10,0.63899,NaN,0,C,10,770,4,3080,0.0
1,230033,1846,0.8095,0.000,0.0,0.0,102,7532,3171,18234,13664,NaN,765,1931,42613,NaN,0,0,0,0,0,74.25,953.06,953.06,4.80,1028,2099,2099,30386,2281,NaN,2251,169,0.4167,0,0,8,0,2,0,99.129,NaN,0,13,0.63836,NaN,NaN,L,732,437,5,2185,1.0
2,230034,1745,0.4001,0.000,0.0,0.0,NaN,2536,NaN,NaN,2536,NaN,NaN,NaN,76109,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,669,4623,3772,NaN,NaN,25.0833,0,NaN,1,NaN,1,0,NaN,29.29,0,1,1.00000,NaN,0,C,89,795,4,3180,1.0
3,230035,1739,0.2193,0.000,0.0,0.0,1982,26440,4955,20316,37013,NaN,0,0,84235,NaN,0,0,0,0,0,NaN,0.00,NaN,0.00,1308,2525,791,91,5992,NaN,3741,215,10.3333,0,0,3,3,2,0,96.272,NaN,0.15385,3,0.53241,0,0,L,3,755,4,3020,0.0
4,230036,1787,0.0118,0.225,0.0,0.0,5451,5494,5494,7987,4696,NaN,2257,27815,123875,524848,0,0,0,0,0,20.51,796.67,NaN,28.72,801,2281,2281,487,2707,NaN,1947,158,0.9167,0,0,2,3,2,0,115.019,NaN,0,1,0.92665,NaN,NaN,L,5,425,4,1700,0.0
5,230037,1579,NaN,3.502,0.0,0.0,NaN,NaN,NaN,NaN,0,199,NaN,NaN,77298,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,183,973,NaN,NaN,NaN,1.3333,0,NaN,2,NaN,2,NaN,NaN,NaN,1.5,0,NaN,NaN,NaN,C,35,455,4,1820,1.0
6,230038,1818,0.4001,0.000,0.0,0.0,NaN,1088,NaN,1536,1498,NaN,448,2331,123875,NaN,NaN,0,0,0,0,NaN,NaN,NaN,29.16,1034,760,NaN,30386,3437,NaN,791,34,9.3333,0,NaN,2,0,0,0,88.171,NaN,0,2,0.87224,NaN,0,C,2,392,5,1960,1.0
7,230039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,198200,1263525,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,2,615,NaN,NaN,0.0
8,230040,1836,0.1358,0.000,0.0,0.0,347,38964,17828,70729,65843,NaN,16298,18877,109010,NaN,0,0,0,1,1,32.02,0.00,0.00,32.92,3180,8943,8943,1490,12075,NaN,NaN,1673,5.9167,0,NaN,6,12,2,0,NaN,NaN,0,10,0.89868,0,0,L,5,821,5,4105,1.0
9,230041,1839,0.1981,0.000,0.0,0.0,793,6131,6045,48959,31640,NaN,10565,14282,99100,146668,0,0,0,1,1,46.17,NaN,NaN,49.07,1701,8912,2707,61

In [11]:
df.reset_index(drop = True, inplace = True)

In [12]:
df.fillna(-1, inplace = True)

In [13]:
def convert_to_int(entry):
    return float(entry)

for col in df.columns:
    if col != 'Type of Product':
        df[col] = df[col].apply(convert_to_int)
    
df['Credit Worthiness Score'][0]

1696.0

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130000 entries, 0 to 129999
Data columns (total 53 columns):
 #   Column                                                                  Non-Null Count   Dtype  
---  ------                                                                  --------------   -----  
 0   ID                                                                      130000 non-null  float64
 1   Credit Worthiness Score                                                 130000 non-null  float64
 2   Number and Riskiness of Credit Enquiry                                  130000 non-null  float64
 3   Default Severity (Any Loan)                                             130000 non-null  float64
 4   Default Severity (Auto Loan)                                            130000 non-null  float64
 5   Default Severity (Education Loan)                                       130000 non-null  float64
 6   Min Cred Available (Revolving Credit Cards)                         

In [15]:
df['default_ind'].value_counts()

 0.0    59145
-1.0    47000
 1.0    23855
Name: default_ind, dtype: int64

In [16]:
le = LabelEncoder()
df['Type of Product'] = le.fit_transform(df['Type of Product'])

In [17]:
import re
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [18]:
train = df.iloc[:83000, :]

X_data = train.drop(['ID', 'default_ind', 'StayDurationofCurrentResidence', 'AnnualIncome', 'TenureofOldestCreditLine'], axis = 1)
y_data = train['default_ind']

#### Optimal Hyperparameter search

In [24]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
rf = RandomForestClassifier(random_state = 0, n_jobs = -1)

def clf_performance(classifier, model_name):
    print(model_name)
    print('Best Score: ' + str(classifier.best_score_))
    print('Best Parameters: ' + str(classifier.best_params_))
    
param_grid =  {'n_estimators': range(100, 200, 10), 
               'criterion':['gini','entropy'],
               'max_depth': [4, 6, 8, 10, 16, 20, 32, 50, None],
               'max_features': ['sqrt', 10, 50],
               'min_samples_leaf': [2,4,8,16,32],
               'min_samples_split': [4,8,16]}

sm = SMOTE(random_state = 0, k_neighbors = 3, n_jobs = -1)
X_Train, Y_Train = sm.fit_resample(X_data, y_data)

clf_rf = RandomizedSearchCV(rf, n_iter=1000, scoring = 'f1', param_distributions = param_grid,
                            cv = 5, verbose = 10, refit = True)

best_clf_rf = clf_rf.fit(X_Train, Y_Train)

Fitting 5 folds for each of 1000 candidates, totalling 5000 fits
[CV 1/5; 1/1000] START criterion=gini, max_depth=8, max_features=sqrt, min_samples_leaf=8, min_samples_split=16, n_estimators=140
[CV 1/5; 1/1000] END criterion=gini, max_depth=8, max_features=sqrt, min_samples_leaf=8, min_samples_split=16, n_estimators=140;, score=0.641 total time=   4.8s
[CV 2/5; 1/1000] START criterion=gini, max_depth=8, max_features=sqrt, min_samples_leaf=8, min_samples_split=16, n_estimators=140
[CV 2/5; 1/1000] END criterion=gini, max_depth=8, max_features=sqrt, min_samples_leaf=8, min_samples_split=16, n_estimators=140;, score=0.648 total time=   4.0s
[CV 3/5; 1/1000] START criterion=gini, max_depth=8, max_features=sqrt, min_samples_leaf=8, min_samples_split=16, n_estimators=140
[CV 3/5; 1/1000] END criterion=gini, max_depth=8, max_features=sqrt, min_samples_leaf=8, min_samples_split=16, n_estimators=140;, score=0.846 total time=   3.7s
[CV 4/5; 1/1000] START criterion=gini, max_depth=8, max_featur

[CV 4/5; 6/1000] END criterion=gini, max_depth=None, max_features=10, min_samples_leaf=4, min_samples_split=4, n_estimators=130;, score=0.914 total time=   9.0s
[CV 5/5; 6/1000] START criterion=gini, max_depth=None, max_features=10, min_samples_leaf=4, min_samples_split=4, n_estimators=130
[CV 5/5; 6/1000] END criterion=gini, max_depth=None, max_features=10, min_samples_leaf=4, min_samples_split=4, n_estimators=130;, score=0.912 total time=   9.0s
[CV 1/5; 7/1000] START criterion=entropy, max_depth=10, max_features=10, min_samples_leaf=2, min_samples_split=4, n_estimators=130
[CV 1/5; 7/1000] END criterion=entropy, max_depth=10, max_features=10, min_samples_leaf=2, min_samples_split=4, n_estimators=130;, score=0.600 total time=   8.2s
[CV 2/5; 7/1000] START criterion=entropy, max_depth=10, max_features=10, min_samples_leaf=2, min_samples_split=4, n_estimators=130
[CV 2/5; 7/1000] END criterion=entropy, max_depth=10, max_features=10, min_samples_leaf=2, min_samples_split=4, n_estimators

[CV 2/5; 12/1000] END criterion=gini, max_depth=6, max_features=10, min_samples_leaf=2, min_samples_split=4, n_estimators=160;, score=0.652 total time=   5.7s
[CV 3/5; 12/1000] START criterion=gini, max_depth=6, max_features=10, min_samples_leaf=2, min_samples_split=4, n_estimators=160
[CV 3/5; 12/1000] END criterion=gini, max_depth=6, max_features=10, min_samples_leaf=2, min_samples_split=4, n_estimators=160;, score=0.835 total time=   5.3s
[CV 4/5; 12/1000] START criterion=gini, max_depth=6, max_features=10, min_samples_leaf=2, min_samples_split=4, n_estimators=160
[CV 4/5; 12/1000] END criterion=gini, max_depth=6, max_features=10, min_samples_leaf=2, min_samples_split=4, n_estimators=160;, score=0.841 total time=   5.3s
[CV 5/5; 12/1000] START criterion=gini, max_depth=6, max_features=10, min_samples_leaf=2, min_samples_split=4, n_estimators=160
[CV 5/5; 12/1000] END criterion=gini, max_depth=6, max_features=10, min_samples_leaf=2, min_samples_split=4, n_estimators=160;, score=0.837

[CV 5/5; 17/1000] END criterion=entropy, max_depth=16, max_features=50, min_samples_leaf=4, min_samples_split=4, n_estimators=150;, score=0.907 total time=  50.2s
[CV 1/5; 18/1000] START criterion=entropy, max_depth=None, max_features=sqrt, min_samples_leaf=8, min_samples_split=8, n_estimators=100
[CV 1/5; 18/1000] END criterion=entropy, max_depth=None, max_features=sqrt, min_samples_leaf=8, min_samples_split=8, n_estimators=100;, score=0.578 total time=   5.5s
[CV 2/5; 18/1000] START criterion=entropy, max_depth=None, max_features=sqrt, min_samples_leaf=8, min_samples_split=8, n_estimators=100
[CV 2/5; 18/1000] END criterion=entropy, max_depth=None, max_features=sqrt, min_samples_leaf=8, min_samples_split=8, n_estimators=100;, score=0.586 total time=   5.6s
[CV 3/5; 18/1000] START criterion=entropy, max_depth=None, max_features=sqrt, min_samples_leaf=8, min_samples_split=8, n_estimators=100
[CV 3/5; 18/1000] END criterion=entropy, max_depth=None, max_features=sqrt, min_samples_leaf=8,

[CV 3/5; 23/1000] END criterion=gini, max_depth=4, max_features=10, min_samples_leaf=4, min_samples_split=16, n_estimators=190;, score=0.800 total time=   4.5s
[CV 4/5; 23/1000] START criterion=gini, max_depth=4, max_features=10, min_samples_leaf=4, min_samples_split=16, n_estimators=190
[CV 4/5; 23/1000] END criterion=gini, max_depth=4, max_features=10, min_samples_leaf=4, min_samples_split=16, n_estimators=190;, score=0.807 total time=   4.5s
[CV 5/5; 23/1000] START criterion=gini, max_depth=4, max_features=10, min_samples_leaf=4, min_samples_split=16, n_estimators=190
[CV 5/5; 23/1000] END criterion=gini, max_depth=4, max_features=10, min_samples_leaf=4, min_samples_split=16, n_estimators=190;, score=0.802 total time=   4.5s
[CV 1/5; 24/1000] START criterion=entropy, max_depth=None, max_features=10, min_samples_leaf=32, min_samples_split=8, n_estimators=150
[CV 1/5; 24/1000] END criterion=entropy, max_depth=None, max_features=10, min_samples_leaf=32, min_samples_split=8, n_estimator

...

[CV 3/5; 976/1000] END criterion=entropy, max_depth=4, max_features=50, min_samples_leaf=32, min_samples_split=8, n_estimators=180;, score=0.770 total time=  20.6s
[CV 4/5; 976/1000] START criterion=entropy, max_depth=4, max_features=50, min_samples_leaf=32, min_samples_split=8, n_estimators=180
[CV 4/5; 976/1000] END criterion=entropy, max_depth=4, max_features=50, min_samples_leaf=32, min_samples_split=8, n_estimators=180;, score=0.780 total time=  20.6s
[CV 5/5; 976/1000] START criterion=entropy, max_depth=4, max_features=50, min_samples_leaf=32, min_samples_split=8, n_estimators=180
[CV 5/5; 976/1000] END criterion=entropy, max_depth=4, max_features=50, min_samples_leaf=32, min_samples_split=8, n_estimators=180;, score=0.789 total time=  20.6s
[CV 1/5; 977/1000] START criterion=entropy, max_depth=10, max_features=sqrt, min_samples_leaf=8, min_samples_split=8, n_estimators=140
[CV 1/5; 977/1000] END criterion=entropy, max_depth=10, max_features=sqrt, min_samples_leaf=8, min_samples_

[CV 1/5; 982/1000] END criterion=gini, max_depth=50, max_features=10, min_samples_leaf=8, min_samples_split=16, n_estimators=110;, score=0.573 total time=   7.3s
[CV 2/5; 982/1000] START criterion=gini, max_depth=50, max_features=10, min_samples_leaf=8, min_samples_split=16, n_estimators=110
[CV 2/5; 982/1000] END criterion=gini, max_depth=50, max_features=10, min_samples_leaf=8, min_samples_split=16, n_estimators=110;, score=0.584 total time=   7.3s
[CV 3/5; 982/1000] START criterion=gini, max_depth=50, max_features=10, min_samples_leaf=8, min_samples_split=16, n_estimators=110
[CV 3/5; 982/1000] END criterion=gini, max_depth=50, max_features=10, min_samples_leaf=8, min_samples_split=16, n_estimators=110;, score=0.900 total time=   7.2s
[CV 4/5; 982/1000] START criterion=gini, max_depth=50, max_features=10, min_samples_leaf=8, min_samples_split=16, n_estimators=110
[CV 4/5; 982/1000] END criterion=gini, max_depth=50, max_features=10, min_samples_leaf=8, min_samples_split=16, n_estimat

[CV 4/5; 987/1000] END criterion=gini, max_depth=16, max_features=sqrt, min_samples_leaf=16, min_samples_split=16, n_estimators=190;, score=0.890 total time=   7.2s
[CV 5/5; 987/1000] START criterion=gini, max_depth=16, max_features=sqrt, min_samples_leaf=16, min_samples_split=16, n_estimators=190
[CV 5/5; 987/1000] END criterion=gini, max_depth=16, max_features=sqrt, min_samples_leaf=16, min_samples_split=16, n_estimators=190;, score=0.887 total time=   7.1s
[CV 1/5; 988/1000] START criterion=entropy, max_depth=10, max_features=50, min_samples_leaf=8, min_samples_split=4, n_estimators=150
[CV 1/5; 988/1000] END criterion=entropy, max_depth=10, max_features=50, min_samples_leaf=8, min_samples_split=4, n_estimators=150;, score=0.552 total time=  38.4s
[CV 2/5; 988/1000] START criterion=entropy, max_depth=10, max_features=50, min_samples_leaf=8, min_samples_split=4, n_estimators=150
[CV 2/5; 988/1000] END criterion=entropy, max_depth=10, max_features=50, min_samples_leaf=8, min_samples_s

[CV 2/5; 993/1000] END criterion=entropy, max_depth=4, max_features=50, min_samples_leaf=16, min_samples_split=8, n_estimators=130;, score=0.614 total time=  16.6s
[CV 3/5; 993/1000] START criterion=entropy, max_depth=4, max_features=50, min_samples_leaf=16, min_samples_split=8, n_estimators=130
[CV 3/5; 993/1000] END criterion=entropy, max_depth=4, max_features=50, min_samples_leaf=16, min_samples_split=8, n_estimators=130;, score=0.770 total time=  15.0s
[CV 4/5; 993/1000] START criterion=entropy, max_depth=4, max_features=50, min_samples_leaf=16, min_samples_split=8, n_estimators=130
[CV 4/5; 993/1000] END criterion=entropy, max_depth=4, max_features=50, min_samples_leaf=16, min_samples_split=8, n_estimators=130;, score=0.780 total time=  14.9s
[CV 5/5; 993/1000] START criterion=entropy, max_depth=4, max_features=50, min_samples_leaf=16, min_samples_split=8, n_estimators=130
[CV 5/5; 993/1000] END criterion=entropy, max_depth=4, max_features=50, min_samples_leaf=16, min_samples_spli

[CV 5/5; 998/1000] END criterion=entropy, max_depth=4, max_features=sqrt, min_samples_leaf=4, min_samples_split=16, n_estimators=150;, score=0.790 total time=   2.8s
[CV 1/5; 999/1000] START criterion=gini, max_depth=4, max_features=10, min_samples_leaf=2, min_samples_split=16, n_estimators=130
[CV 1/5; 999/1000] END criterion=gini, max_depth=4, max_features=10, min_samples_leaf=2, min_samples_split=16, n_estimators=130;, score=0.672 total time=   3.3s
[CV 2/5; 999/1000] START criterion=gini, max_depth=4, max_features=10, min_samples_leaf=2, min_samples_split=16, n_estimators=130
[CV 2/5; 999/1000] END criterion=gini, max_depth=4, max_features=10, min_samples_leaf=2, min_samples_split=16, n_estimators=130;, score=0.676 total time=   3.3s
[CV 3/5; 999/1000] START criterion=gini, max_depth=4, max_features=10, min_samples_leaf=2, min_samples_split=16, n_estimators=130
[CV 3/5; 999/1000] END criterion=gini, max_depth=4, max_features=10, min_samples_leaf=2, min_samples_split=16, n_estimator

In [25]:
clf_performance(best_clf_rf,'Random Forest')

Random Forest
Best Score: 0.783254866569141
Best Parameters: {'n_estimators': 180, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 32, 'criterion': 'gini'}


Best Parameters: {'n_estimators': 180, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 32, 'criterion': 'gini'}

In [64]:
kfold = StratifiedKFold(random_state = 10, n_splits = 5, shuffle = True)
splits = kfold.split(X_data, y_data)

sm = SMOTE(random_state = 10, k_neighbors = 3, n_jobs = -1)

xgb_acc_scores = []
xgb_f1_scores = []
xgb_models = []

rf_acc_scores = []
rf_f1_scores = []
rf_models = []

i=1

for Train, Test in splits:
    X_Train, X_Test, Y_Train, Y_Test = X_data.iloc[Train], X_data.iloc[Test], y_data.iloc[Train], y_data.iloc[Test]
    
    #xgb = XGBClassifier(n_estimators = 2000, max_depth = 8, n_jobs = -1, reg_lambda = 0.1,
    #                    learning_rate = 0.05, random_state = 0, early_stopping_rounds = 200)
    
    rf = RandomForestClassifier(n_estimators = 120, max_depth=8, n_jobs=-1, random_state=10)

    X_Train, Y_Train = sm.fit_resample(X_Train, Y_Train)
    
    '''xgb.fit(X_Train, Y_Train, eval_set = [(X_Train, Y_Train), (X_Test, Y_Test)], verbose = True)
    
    pred = xgb.predict(X_Test)
    
    xgb_f1_scores.append(f1_score(Y_Test, pred))
    xgb_acc_scores.append(accuracy_score(Y_Test, pred)*100)
    xgb_models.append(xgb)
    '''
    rf.fit(X_Train, Y_Train)
    
    pred = rf.predict(X_Test)
    
    rf_f1_scores.append(f1_score(Y_Test, pred))
    rf_acc_scores.append(accuracy_score(Y_Test, pred)*100)
    rf_models.append(rf)
    
    print("Iteration {} Completed".format(i))
    i+=1

Iteration 1 Completed
Iteration 2 Completed
Iteration 3 Completed
Iteration 4 Completed
Iteration 5 Completed


In [65]:
#print("XGB F1 Score: ", sum(xgb_f1_scores)/len(xgb_f1_scores))
#print("XGB Accuracy: ", sum(xgb_acc_scores)/len(xgb_acc_scores), "%")
print("RF F1 Score: ", sum(rf_f1_scores)/len(rf_f1_scores))
print("RF Accuracy: ", sum(rf_acc_scores)/len(rf_acc_scores), "%")

RF F1 Score:  0.5748584894456157
RF Accuracy:  74.22409638554217 %


In [ ]:
'''
0.575542
rf = RandomForestClassifier(n_estimators = 140, max_depth=8, n_jobs=-1, random_state=10)
model = xgb_models[0]

plt.figure(figsize = (10,10))
plot_importance(model, max_num_features = 5)
plt.show()'''

In [66]:
test = df.iloc[83000:, :]
test = test.drop(['ID', 'default_ind', 'StayDurationofCurrentResidence', 'AnnualIncome', 'TenureofOldestCreditLine'], axis = 1)

In [67]:
preds = np.argmax(#xgb_models[0].predict_proba(test) +
                  #xgb_models[1].predict_proba(test) +
                  #xgb_models[2].predict_proba(test) +
                  #xgb_models[3].predict_proba(test) +
                  #xgb_models[4].predict_proba(test) +
                  rf_models[0].predict_proba(test) +
                  rf_models[1].predict_proba(test) +
                  rf_models[2].predict_proba(test) +
                  rf_models[3].predict_proba(test) +
                  rf_models[4].predict_proba(test), axis = 1)

In [31]:
preds = best_clf_rf.best_estimator_.predict(test)

In [68]:
sample = pd.read_csv('submission.csv', header = None)
sample.head()

,0,1
0,578069,0
1,578070,1
2,578071,1
3,578072,1
4,578073,0


In [69]:
sample[1] = preds
sample.head()

,0,1
0,578069,0
1,578070,0
2,578071,0
3,578072,1
4,578073,0


In [70]:
sample[1].value_counts()

0    31404
1    15596
Name: 1, dtype: int64

In [71]:
#sample.to_csv('Submissions/ThirdDegreeBurn_9.csv', index = False, header=False)